# Student Database Management System

In [1]:
#importing necessary libraries
from pymongo import MongoClient
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
from tkinter import font
from tkinter import Label

# MongoDB connection setup
client = MongoClient('mongodb://localhost:27017/')
db = client['StudentDatabaseMangementSystem']
students_collection = db['students_info']

# Function to fetch data from selected collection and display it
def fetch_data():
    selected_collection = collection_var.get()
    if selected_collection:
        collection = db[selected_collection]
        documents = collection.find()
        update_treeview_columns(documents)
    else:
        messagebox.showerror("Error", "Please select a collection.")

# Update Treeview columns based on the documents of the selected collection
def update_treeview_columns(documents):
    treeview.delete(*treeview.get_children())
    if documents.count() > 0:
        first_doc = documents[0]
        fields = list(first_doc.keys())
        treeview['columns'] = fields
        for field in fields:
            treeview.heading(field, text=field)
            treeview.column(field, width=100, anchor='center')
        # Display all documents
        for doc in documents:
            values = [str(doc[field]) for field in fields]
            treeview.insert("", 'end', values=values)
    else:
        messagebox.showinfo("Information", "No documents found.")

# Function to register a student
def register_student():
    if students_collection.find_one({"FirstName": firstname_entry.get(), "LastName": lastname_entry.get()}):
        messagebox.showerror("Error", "Student with this name already exists!")
    else:
        try:
            max_id_doc = students_collection.find_one(sort=[("_id", -1)])
            next_id = 1 if not max_id_doc else max_id_doc["_id"] + 1

            student_info = {
                "_id": next_id,
                "FirstName": firstname_entry.get(),
                "LastName": lastname_entry.get(),
                "DateOfBirth": dob_entry.get(),
                "ContactNumber": contactnumber_entry.get(),
                "Email": email_entry.get(),
            }
            students_collection.insert_one(student_info)
            messagebox.showinfo("Success", "Student registered successfully.")
            display_all_students()
            clear_entries()
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

# Function to search for a student
def search_student():
    search_value = search_entry.get()
    result = students_collection.find({"$or": [
        {"FirstName": {"$regex": search_value, "$options": "i"}},
        {"LastName": {"$regex": search_value, "$options": "i"}}
    ]})
    treeview.delete(*treeview.get_children())
    
    # Configure Treeview columns for search results
    treeview['columns'] = ("ID", "First Name", "Last Name", "Date of Birth", "Contact", "Email")
    treeview.heading("ID", text="ID")
    treeview.heading("First Name", text="First Name")
    treeview.heading("Last Name", text="Last Name")
    treeview.heading("Date of Birth", text="Date of Birth")
    treeview.heading("Contact", text="Contact")
    treeview.heading("Email", text="Email")
    
    # Set column widths and alignment
    treeview.column("ID", anchor='center', width=50)
    treeview.column("First Name", anchor='w', width=100)
    treeview.column("Last Name", anchor='w', width=100)
    treeview.column("Date of Birth", anchor='w', width=100)
    treeview.column("Contact", anchor='w', width=100)
    treeview.column("Email", anchor='w', width=200)
    
    # Insert search results into the Treeview
    for student in result:
        treeview.insert("", 'end', values=(
            student.get("_id", ''),
            student.get("FirstName", ''),
            student.get("LastName", ''),
            student.get("DateOfBirth", ''),
            student.get("ContactNumber", ''),
            student.get("Email", '')
        ))

# Function to display all students
def display_all_students():
    result = students_collection.find()
    treeview.delete(*treeview.get_children())  # Clear existing entries

    # Configure the Treeview columns
    treeview['columns'] = ("ID", "First Name", "Last Name", "Date of Birth", "Contact", "Email")
    treeview.heading("ID", text="ID")
    treeview.heading("First Name", text="First Name")
    treeview.heading("Last Name", text="Last Name")
    treeview.heading("Date of Birth", text="Date of Birth")
    treeview.heading("Contact", text="Contact")
    treeview.heading("Email", text="Email")
    
    # Set column widths and alignment
    treeview.column("ID", anchor='center', width=50)
    treeview.column("First Name", anchor='w', width=100)
    treeview.column("Last Name", anchor='w', width=100)
    treeview.column("Date of Birth", anchor='w', width=100)
    treeview.column("Contact", anchor='w', width=100)
    treeview.column("Email", anchor='w', width=200)
    
    # Insert results into the Treeview
    for student in result:
        treeview.insert("", 'end', values=(
            student.get("_id", ''),
            student.get("FirstName", ''),
            student.get("LastName", ''),
            student.get("DateOfBirth", ''),
            student.get("ContactNumber", ''),
            student.get("Email", '')
        ))

# Function to delete a student
def delete_student():
    selected_item = treeview.selection()
    if selected_item:
        selected_student = treeview.item(selected_item[0])['values']
        student_id = int(selected_student[0])
        students_collection.delete_one({"_id": student_id})
        treeview.delete(selected_item[0])
        messagebox.showinfo("Success", "Student deleted successfully.")
    else:
        messagebox.showerror("Error", "Please select a student to delete.")

## Function to update a student record
def update_student():
    selected_item = treeview.selection()
    if selected_item:
        update_window = tk.Toplevel(root)
        update_window.title("Update Student Information")

        selected_student_id = treeview.item(selected_item[0])['values'][0]
        student_id = int(selected_student_id)
        student = students_collection.find_one({"_id": student_id})

        # Fields for editable first and last names
        ttk.Label(update_window, text="First Name:", font=label_font, foreground=label_color).grid(row=0, column=0)
        firstname_update_entry = ttk.Entry(update_window)
        firstname_update_entry.grid(row=0, column=1, padx=10, pady=5)
        firstname_update_entry.insert(0, student["FirstName"])
        
        ttk.Label(update_window, text="Last Name:", font=label_font, foreground=label_color).grid(row=1, column=0)
        lastname_update_entry = ttk.Entry(update_window)
        lastname_update_entry.grid(row=1, column=1, padx=10, pady=5)
        lastname_update_entry.insert(0, student["LastName"])

        # Fields for editable contact number and email
        ttk.Label(update_window, text="Contact Number:", font=label_font, foreground=label_color).grid(row=2, column=0)
        contactnumber_update_entry = ttk.Entry(update_window)
        contactnumber_update_entry.grid(row=2, column=1, padx=10, pady=5)
        contactnumber_update_entry.insert(0, student["ContactNumber"])

        ttk.Label(update_window, text="Email:", font=label_font, foreground=label_color).grid(row=3, column=0)
        email_update_entry = ttk.Entry(update_window)
        email_update_entry.grid(row=3, column=1, padx=10, pady=5)
        email_update_entry.insert(0, student["Email"])
        
        # Button to submit the updates
        update_button = ttk.Button(update_window, text="Update", command=lambda: submit_update(update_window, student, firstname_update_entry, lastname_update_entry, contactnumber_update_entry, email_update_entry))
        update_button.grid(row=4, column=0, columnspan=2, pady=10)
    
def submit_update():
            # Here, you should include the logic to update the student's information in the database.
            # For now, it's a stub that always returns True.
            # In your application, it should return True if the update succeeds and False if it fails.
    success = True  # Placeholder for actual update logic
    
            # Placeholder for the update operation; replace with actual update logic.
    success = update_student_information(student, firstname_entry.get(), lastname_entry.get(), contactnumber_entry.get(), email_entry.get())

            # Close the update window after updating
    update_window.destroy()

    # Show a success or failure message
    if success:
        messagebox.showinfo("Update Successful", "The student record has been successfully updated.")
    else:
        messagebox.showwarning("No Selection", "Please select a student to update.")

def submit_update(update_window, student, firstname_update_entry, lastname_update_entry, contactnumber_update_entry, email_update_entry):
    students_collection.update_one(
        {"_id": student["_id"]},
        {
            "$set": {
                "FirstName": firstname_update_entry.get(),
                "LastName": lastname_update_entry.get(),
                "ContactNumber": contactnumber_update_entry.get(),
                "Email": email_update_entry.get(),
            }
        }
    )
    update_window.destroy()
    display_all_students()

# Function to clear input entries
def clear():
    search_entry.delete(0,tk.END)
    firstname_entry.delete(0, tk.END)
    lastname_entry.delete(0, tk.END)
    dob_entry.delete(0, tk.END)
    contactnumber_entry.delete(0, tk.END)
    email_entry.delete(0, tk.END)

# Create the main GUI window
root = tk.Tk()
#root.title("Student Database Management System")

# Add background image
#background_image = Image.open("C:/Users/user/OneDrive/Desktop/background_image.jpg")
#background_photo = ImageTk.PhotoImage(background_image)
#background_label = Label(root,image=background_photo)
#background_label.place(relwidth=1, relheight=1)

# Define custom fonts and styles
#heading_font = font.Font(family="Helvetica", size=18, weight="bold", root=root)
#label_font = font.Font(family="Arial", size=12, root=root)

# Defining custom fonts and styles
heading_font = font.Font(family="Times New Roman", size=18, weight="bold", root=root)
label_font = font.Font(family="Times New Roman", size=12, root=root)
label_color = "#2B2B2B"


# Define frames for layout
top_frame = ttk.Frame(root)
top_frame.pack(fill=tk.X, padx=10, pady=10)
bottom_frame = ttk.Frame(root)
bottom_frame.pack(fill=tk.X, padx=10, pady=10)

# Heading label with custom font and color
heading_label = ttk.Label(top_frame, text="Student Database Management System", font=heading_font, foreground="#00457C")
heading_label.grid(row=0, columnspan=3, pady=10)

# Labels and input fields for student registration
ttk.Label(top_frame, text="First Name:", font=label_font, foreground=label_color).grid(row=1, column=0, sticky='w')
firstname_entry = ttk.Entry(top_frame)
firstname_entry.grid(row=1, column=1, padx=10, pady=5)

ttk.Label(top_frame, text="Last Name:", font=label_font, foreground=label_color).grid(row=2, column=0, sticky='w')
lastname_entry = ttk.Entry(top_frame)
lastname_entry.grid(row=2, column=1, padx=10, pady=5)

ttk.Label(top_frame, text="Date of Birth:", font=label_font, foreground=label_color).grid(row=3, column=0, sticky='w')
dob_entry = ttk.Entry(top_frame)
dob_entry.grid(row=3, column=1, padx=10, pady=5)

ttk.Label(top_frame, text="Contact Number:", font=label_font, foreground=label_color).grid(row=4, column=0, sticky='w')
contactnumber_entry = ttk.Entry(top_frame)
contactnumber_entry.grid(row=4, column=1, padx=10, pady=5)

ttk.Label(top_frame, text="Email:", font=label_font, foreground=label_color).grid(row=5, column=0, sticky='w')
email_entry = ttk.Entry(top_frame)
email_entry.grid(row=5, column=1, padx=10, pady=5)

# Buttons for student registration, display, and search
register_button = ttk.Button(top_frame, text="Register Student", command=register_student)
register_button.grid(row=6, column=0, columnspan=2, pady=10)

search_entry = ttk.Entry(top_frame)
search_entry.grid(row=7, column=0, padx=10, pady=5, columnspan=2)

search_button = ttk.Button(top_frame, text="Search Student", command=search_student)
search_button.grid(row=7, column=2, padx=10, pady=5)

display_all_button = ttk.Button(top_frame, text="Display All Students", command=display_all_students)
display_all_button.grid(row=8, column=0, columnspan=2, pady=10)

# Treeview for displaying student information
treeview = ttk.Treeview(bottom_frame, columns=("ID", "First Name", "Last Name", "Date of Birth", "Contact", "Email"), show='headings')
treeview.heading("ID", text="ID")
treeview.heading("First Name", text="First Name")
treeview.heading("Last Name", text="Last Name")
treeview.heading("Date of Birth", text="Date of Birth")
treeview.heading("Contact", text="Contact")
treeview.heading("Email", text="Email")

# Set column widths and alignment for better presentation
treeview.column("ID", anchor='center', width=50)
treeview.column("First Name", anchor='w', width=100)
treeview.column("Last Name", anchor='w', width=100)
treeview.column("Date of Birth", anchor='w', width=100)
treeview.column("Contact", anchor='w', width=100)
treeview.column("Email", anchor='w', width=200)

treeview.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

# Frame for delete update, clear buttons
button_frame = ttk.Frame(bottom_frame)
button_frame.pack(fill=tk.X, padx=10, pady=10)

delete_button = ttk.Button(button_frame, text="Delete Student", command=delete_student)
delete_button.pack(side='left', padx=10, pady=10)

update_button = ttk.Button(button_frame, text="Update Student", command=update_student)
update_button.pack(side='left', padx=10, pady=10)

#Clear button
clear_button = ttk.Button(root, text="Clear", command=clear)
clear_button.pack(side='right', padx=10, pady=10)


# Run the application
root.mainloop()